<a href="https://colab.research.google.com/github/arvindravulavaru/scraper/blob/main/Shopify_dev_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install html2text


In [76]:
import requests
from bs4 import BeautifulSoup
import html2text
import datetime
import pytz
import os
import tarfile
from urllib.parse import urlparse, parse_qs, urljoin

now = datetime.datetime.now(pytz.timezone('US/Eastern')).strftime("%Y-%m-%d %H:%M:%S")

def remove_tags(soup, tag_names):
    """
    This function removes specified tags from HTML.

    Parameters:
    soup (bs4.BeautifulSoup) : BeautifulSoup object
    tag_names (list) : list of strings

    Returns:
    str : cleaned HTML
    """

    # Remove specified tags from HTML
    for tag_name in tag_names:
        for tag in soup.find_all(tag_name):
            tag.decompose()

    # Return cleaned HTML
    return str(soup)


# Function to scrape links recursively
def scrape_links(url, folder_path, max_files, scraped_links, tag_names):
    """
    This function scrapes links recursively, writes content to files, and returns scraped links.

    Parameters:
    url (str) : string representing URL to scrape
    folder_path (str) : string representing folder path where scraped files are stored
    max_files (int) : maximum number of files to scrape
    scraped_links (list) : list of strings representing URLs already scraped
    tag_names (list) : list of strings representing tags to remove from HTML

    Returns:
    list : list of strings representing scraped URLs
    """
    
    # Check if we have reached the maximum number of files
    if len(scraped_links) >= max_files:
        return scraped_links

    # Write content to files
    parsed_url = urlparse(url)

    # Send GET request to URL if it is from the shopify domain
    if "shopify.dev" not in urlparse(url).netloc:
        return scraped_links
    
    print(f'Processing URL: [{len(scraped_links) + 1}] {url}')
    response = requests.get(url)

    # Parse HTML content
    soup = BeautifulSoup(response.content, 'html.parser')

    # Remove specified tags from HTML
    page_html = remove_tags(soup, tag_names)

    # Convert HTML to Markdown
    converter = html2text.HTML2Text()
    converter.body_width = 0
    page_markdown = converter.handle(page_html)

    # Extract text content from HTML
    page_text = soup.get_text()
    
    # query_string = parse_qs(parsed_url.query, keep_blank_values=True)
    # query_string_formatted = "_".join([f"{k}={v[0]}" for k, v in query_string.items()]) if query_string else ""

    file_name = parsed_url.path[1:].replace('/', '-')
    file_name = file_name.replace('https-', '').replace('www-', '').replace(':', '').replace('.', '-')

    # Create directory if it doesn't exist
    file_path = os.path.join(folder_path, file_name)
    if not os.path.exists(file_path):
        os.makedirs(file_path)

    # Write HTML content to file
    with open(f'{file_path}/index.html', 'w') as f:
        f.write(f"<!-- Generated on: {now} EST -->\n")
        f.write(f"<!-- Scraped URL: {url} -->\n")
        f.write(page_html)

    # Write Markdown content to file
    with open(f'{file_path}/index.md', 'w') as f:
        f.write(f"<!-- Generated on: {now} EST -->\n")
        f.write(f"<!-- Scraped URL: {url} -->\n")
        f.write(page_markdown)

    # Write text content to file
    with open(f'{file_path}/index.txt', 'w') as f:
        f.write(f"<!-- Generated on: {now} EST -->\n")
        f.write(f"<!-- Scraped URL: {url} -->\n")
        f.write(page_text)

    # Add link to list of scraped links
    scraped_links.append(url)

    # Recursively scrape links on the page
    for link in soup.find_all('a'):
        href = link.get('href')
        if href is not None:
            href = urljoin(url, href) # convert to absolute URL
            
            if href.startswith('http') and '#' not in href and href not in scraped_links:
                scrape_links(href, folder_path, max_files, scraped_links, tag_names)
                
                # Check if we have reached the maximum number of files
                if len(scraped_links) >= max_files:
                    return scraped_links

    return scraped_links


# Scrape Shopify Developer Documentation website
url = "https://shopify.dev/docs/apps"
# make sure path exists on drive before running the code!
folder_path = '/content/drive/MyDrive/Experiments/Data/shopify-dev/'
# parent_dir = os.path.abspath(os.path.join(folder_path, os.pardir))
parent_dir = '/content/drive/MyDrive/Experiments/Data/'
max_files = 1
tag_names = ['script', 'style', 'link']
tar_filename = 'shopify-dev-files.'+ now + '.tar.gz'
sitemap_filename = 'site_hierarchy.txt'

# Scrape links recursively
scraped_links = scrape_links(url, folder_path, max_files, [], tag_names)

# Print success message
print(f'Successfully wrote {len(scraped_links)} files to Google Drive')
# Write site hierarchy to file
# with open(os.path.join(os.path.dirname(folder_path), f'{sitemap_filename}'), 'w') as f:
#     f.write(f'Site hierarchy:\n<!-- Generated on: {now} EST -->\n\n')
#     for root, dirs, files in os.walk(folder_path):
#         level = root.replace(folder_path, '').count(os.sep)
#         indent = ' ' * 2 * (level)
#         f.write(f'{indent}{os.path.basename(root)}/\n')
#         subindent = ' ' * 2 * (level + 1)
#         for fx in files:
#             f.write(f'{subindent}{fx}\n')

print(os.path.join(os.path.dirname(parent_dir), f'{tar_filename}'))

# create a tarball of all files in the folder_path directory
with tarfile.open(os.path.join(os.path.dirname(parent_dir), f'{tar_filename}'), 'w:gz') as tar:
    for file in os.listdir(folder_path):
        tar.add(os.path.join(folder_path, file), arcname=file)

print(f'Successfully created tarball: {tar_filename}')

Processing URL: [1] https://shopify.dev/docs/apps
Successfully wrote 1 files to Google Drive
/content/drive/MyDrive/Experiments/Data/shopify-dev-files.2023-03-18 11:11:54.tar.gz
Successfully created tarball: shopify-dev-files.2023-03-18 11:11:54.tar.gz
